# Data Cleaning and Features Engineering (Part 2c)
### Clean and Extract Features from Box Office Data (boxofficemojo.com)

In [1]:
import pandas as pd

In [2]:
# Read box office data csv from hive
box_df = pd.read_csv(r".\full_movie_box_office_5000.csv", sep =',')
box_df.head(5)

,movie_title,domestic_gross,domestic_opening,foreign_gross,markets,mpaa,total_gross
0,Toy Story 4,"$433,833,761","$120,908,065","$637,539,788",56 markets,G,"$1,071,373,549"
1,Pretenders,NaN,NaN,"$3,509",Lithuania,NaN,"$3,509"
2,The Lion King,"$543,265,323","$191,770,759","$1,110,039,326",56 markets,NaN,"$1,653,304,649"
3,Dilili in Paris (Dilili à Paris),NaN,NaN,"$4,856,037","APAC, EMEA",NaN,"$4,856,037"
4,Stuber,"$22,370,452","$8,225,384","$10,020,493",34 markets,R,"$32,390,945"


In [3]:
# Define function to obtain specific data from strings
def get_markets(x):
    for x in str.split(x):
        if x.isdigit():
            return int(x)
        else: 
            return None

### Data Cleaning
- Convert columns such as domestic_gross, domestic_opening, foreign_gross, total_gross to numeric
- Extract the number of markets' exposure (numeric) from the "markets" columns (string).
- Calculate the total_gross = domestic_gross + foreign_gross, and to replace missing values in total_gross


In [4]:
# Clean the data accordingly, convert string to appropriate data type (integer/date etc.)
box_df["domestic_gross"] = pd.to_numeric(box_df["domestic_gross"].fillna(0).apply(lambda x: str(x).replace("$", "").replace(",","")), errors = "coerce")
box_df["domestic_opening"] = pd.to_numeric(box_df["domestic_opening"].fillna(0).apply(lambda x: str(x).replace("$", "").replace(",","")), errors = "coerce")
box_df["foreign_gross"] = pd.to_numeric(box_df["foreign_gross"].fillna(0).apply(lambda x: str(x).replace("$", "").replace(",","")), errors = "coerce")
box_df["total_gross"] = pd.to_numeric(box_df["total_gross"].fillna(0).apply(lambda x: str(x).replace("$", "").replace(",","")), errors = "coerce")
box_df["markets"] = box_df["markets"].apply(lambda x: get_markets(x))
# Recalculate total_gross as the sum of domestic_gross and foreign_gross as the original total_gross is not consistent.
box_df["total_gross_calc"] = box_df["domestic_gross"] + box_df["foreign_gross"]
box_df["total_gross"] = box_df["total_gross"].fillna(box_df["total_gross_calc"])

In [5]:
# Drop columns that are not useful
box_df = box_df.drop(["total_gross_calc", "mpaa"], axis = 1)

### Add a column to indicate missingness
- Create a column called 'markets_missing' to indicate missing value in 'markets'

In [10]:
box_df["markets_missing"] = box_df["markets"].isna()*1


### Features 3: Box Office Data from boxofficemojo.com (box_df)


In [12]:
box_df.to_csv("features_box_office.csv", index = False)

In [13]:
box_df.head()

,movie_title,domestic_gross,domestic_opening,foreign_gross,markets,total_gross,markets_missing
0,Toy Story 4,433833761,120908065,637539788,56.0,1071373549,0
1,Pretenders,0,0,3509,NaN,3509,1
2,The Lion King,543265323,191770759,1110039326,56.0,1653304649,0
3,Dilili in Paris (Dilili à Paris),0,0,4856037,NaN,4856037,1
4,Stuber,22370452,8225384,10020493,34.0,32390945,0
